# Introduction to spaCy

Copyright, Vrije Universiteit Amsterdam, Faculty of Humanities, CLTL

Whereas NLTK contains separate modules that need to be called individually, we now look at another toolkit that hides these steps in one processing module. Basically, the toolkit takes the plain text as input, applies a whole series of NLP modules to the text and outputs a complex object with the output of these modules stacked on top of the elements of the text.

[spaCy](https://spacy.io/) provides a rather complete NLP pipeline (the output of one module feeds to the input of the next): it takes a raw document and performs tokenization, POS-tagging, stop word recognition, morphological analysis, lemmatization, sentence splitting, dependency parsing and Named Entity Recognition (NER), among others. It also supports similarity prediction, but that is outside of the scope of this notebook. The advantage of spaCy is that it is really fast, and it has a reasonable accuracy. In addition, it currently supports multiple languages: https://spacy.io/models.

So whereas in NLTK you built your own pipeline, spaCy is for lazy people and can do it all for you. That is not completely correct as you can build your own pipeline also in spaCy but that is not so trivial and most people are lazy.

In this notebook, we will show you the basic usage. If you want to learn more, please visit spaCy's website; it has extensive documentation and provides excellent user guides. 

**At the end of this notebook, you will be able to extract the output from spaCy for the following NLP tasks**:
* **Sentence splitting**: attribute **sents** of a `Doc` (of type *spacy.tokens.doc.Doc*)
* **Tokenization**: `Doc` contains a sequence of `Token` objects (of type *spacy.tokens.token.Token*)
* **Part-of-speech (POS) tagging**: attributes **pos_** and **tag_** of `Token`
* **Stop words recognition** attribute **is_stop** of `Token`
* **Stemming and lemmatization**: attribute **lemma_** of `Token`
* **Constituency/dependency parsing:** attributes **dep_** and **head**
* **Named Entity Recognition (NER):** attribute **ents** (of type *spacy.tokens.span.Span*) of `Doc` (of type *spacy.tokens.doc.Doc*). 

In addition, you will be able to use spaCy to visualize the output for each NLP task.


## Installing and loading spaCy

To install spaCy, check out the instructions [here](https://spacy.io/usage). It explains exactly how to install spaCy for your operating system, package manager and desired languages. Simply run the suggested commands in your terminal ([Anaconda Prompt](https://docs.anaconda.com/anaconda/user-guide/getting-started/) or cmd). Alternatively, you can probably also just run the following cells in this notebook:

**Tip**: comment out the next two commands after using them. You can comment out commands by putting a "#" in front.

In [1]:
%%bash
conda install -c conda-forge spacy

Solving environment: ...working... done

## Package Plan ##

  environment location: /Users/piek/opt/anaconda3

  added / updated specs:
    - spacy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    conda-4.10.3               |   py37hf985489_0         3.0 MB  conda-forge
    ------------------------------------------------------------
                                           Total:         3.0 MB

The following packages will be SUPERSEDED by a higher-priority channel:

  conda              pkgs/main::conda-4.10.3-py37hecd8cb5_0 --> conda-forge::conda-4.10.3-py37hf985489_0


Proceed ([y]/n)? 

conda-4.10.3         | 3.0 MB    | ########## | 100% 
Preparing transaction: ...working... done
Verifying transaction: ...working... done
Executing transaction: ...working... done


If there are no errors, you installed the software on your local machine. Congratulations!  But this is not enough. You also need to have the language resources to feed the software: trained models, grammars, lexicons, etc. spaCy comes with language resources for many different languages and this is growing. Maybe one day you may even contribute a module for your own language.

In this notebook, we are going to download the English language resources. The standard donwload command from the command line is the following:

In [2]:
%%bash
python -m spacy download en

✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
✔ Linking successful
/Users/piek/opt/anaconda3/lib/python3.7/site-packages/en_core_web_sm -->
/Users/piek/opt/anaconda3/lib/python3.7/site-packages/spacy/data/en
You can now load the model via spacy.load('en')


You should consider upgrading via the '/Users/piek/opt/anaconda3/bin/python -m pip install --upgrade pip' command.


In the messages, you see that there is a download and install operation followed by a linking operation. There may be errors with the linking, which we try to solve below.

Now, let's first load spaCy in the notebook and check if we can load the English language resources. We import the spaCy module and load the English tokenizer, tagger, parser, NER, and word vectors.

In [1]:
import spacy
nlp= spacy.load('en') # other languages: de, es, pt, fr, it, nl

If there is no message, you can assume that everything went well and you can skip the next part on errors.

You may want to check the models that are installed for spaCy for their compatibility with the spaCy version that you are running.
The next cell shows you how to do this on the command line:

In [2]:
%%bash
python -m spacy validate

✔ Loaded compatibility table

====================== Installed models (spaCy v2.1.8) ======================
ℹ spaCy installation:
/Users/piek/opt/anaconda3/lib/python3.7/site-packages/spacy

TYPE      NAME              MODEL             VERSION                            
package   nl-core-news-sm   nl_core_news_sm   2.1.0   ✔
package   en-core-web-sm    en_core_web_sm    2.1.0   ✔
link      en                en_core_web_sm    2.1.0   ✔



In my case, there are two models and a link, all compatible with my spaCy. If there is no model listed, something went wrong with downloading. If spaCy lists a compatibility problem, it suggests how to fix it. Below are some more possible fixes. Note that these toolkits are improved rapidly and a new release may change things.

### Possible error when downloading language models

You might get a variation of the following error:
```
OSError: [E050] Can't find model 'en'. It doesn't seem to be a shortcut link, a Python package or a valid path to a data directory.
```

This means that there is a problem with linking the language model of spaCy. You can try to load it in the following way instead:

In [3]:
nlp = spacy.load('en_core_web_sm')

Even if this works, you might still want to invest some time to make sure that the linking was succesful, i.e., that you can load spaCy with spacy.load('en'). Here is some more information on how to fix this.

**Troubleshooting (optional)**

*Cause*: Anaconda prompt does not have enough priviliges to execute the linking part of `python -m spacy download en`. The same is true for any other `python -m spacy [...]` command.

*Solution:* 

1. Create the link manually

The prompt should display something along the lines of:

```
<Data Downloaded>
You do not have sufficient privilege to perform this operation.

    Linking successful
    <Anaconda dir>\lib\site-packages\en_core_web_sm -->
    <Anaconda dir>\lib\site-packages\spacy\data\en

    You can now load the model via spacy.load('en')

Use the following command:

mklink /D <Anaconda>\lib\site-packages\spacy\data\en <Anaconda>\lib\site-packages\en_core_web_sm

 Note that the target is pointing to the link, not the other way around. The syntax of the arguments is 'target' followed by 'link'
 
```

2. Give Anaconda Permissions to create link. Using "runas ... python -m spacy ..." may not suffice

3. More details: https://github.com/explosion/spaCy/issues/1283

**If none of this works (can happen on Windows)** You might want to install the model manually from SpaCy's GitHub through pip:

```
    pip install https://github.com/explosion/spacy-models/releases/download/en_core_web_sm-2.1.0/en_core_web_sm-2.1.0.tar.gz`.
```

Still having problems? Come to the VU feedback sessions or schedule a zoom meeting with the teachers to fix it.

## Using spaCy

If you succesfully loaded the English model (or another language) with the above command, you created the spaCy object 'nlp'. 

An 'object'?????

So what is an object? You can see it as an instance of a class of things. Whenever you create a variable and assign it a value of some type, this variable is a 'typed' instance. Like a baby that is born is a new instance of the type human and a kitten is a new instance of the type cat. It has all the properties of this type and you can do the things with the variable that were defined for that type.

So 'nlp' is an instance of a spaCy object loaded with the models for the English language. This means that 'nlp' has all the functions that the spaCy developers defined.

You can use 'nlp' now to process text through a defined pipeline of modules and store the result as a value for another variable for accessing it. The result of processing a text with spaCy is another spaCy object of the type 'Doc'.

'Doc' objects are complex and give you access to different analyses that have been applied to the input text. In a Doc object you can access tokens that make up the text, their lemmas, their PoS, the sentences, chunks, named entities, and many more. Below, we will look at all the types of analyses in more detail.

To process a text, you simple pass the string to the *nlp* object that we created as input and we assign the result of it to a variable *doc* with a lower case. The variable doc should then be of the tpe Doc. Let's check that.


In [4]:
doc = nlp("I have an awesome cat. She follows me through the house everywhere I go. Her name is shadow")
type(doc)

spacy.tokens.doc.Doc

So that's it. spaCy has now processed the text in many different ways ang gives you the sentences, tokens, dependency relations, named entities, etc.

So whereas we had to call all these specific functions in the right order in the case of NLTK, i.e. sentence_splitting, tokenization, part-of-speech tagging, parsing and named entity recognition, spaCy did all of that when processing a text. The only thing you need to do is to access the output, which is what we will do next.

`doc` is now a Python object of the class `Doc`. It is a container for accessing linguistic annotations and a sequence of `Token` objects. 

#### Doc, Token and Span objects

At this point, there are three important types of objects to remember:

* A `Doc` is a sequence of `Token` objects.
* A `Token` object represents an individual token — i.e. a word, punctuation symbol, whitespace, etc. It has attributes representing linguistic annotations. 
* A `Span` object is a slice from a `Doc` object and consists of a sequence of `Token` objects.

Since `Doc` is a sequence of `Token` objects, we can iterate over all of the tokens in the text as shown below, or select a single token from the sequence. We need a 'for-loop' to iterate over the elements in the doc object just as we would do for a list:

In [28]:
# Iterate over the tokens using a for loop
for token in doc:
    print(token)
print()


I
have
an
awesome
cat
.
She
follows
me
through
the
house
everywhere
I
go
.
Her
name
is
shadow



Note that spaCy does not really create a list but a so-called 'generator'. A generator is a so-called 'lazy iterator' in Python that does not overload memory:

https://realpython.com/introduction-to-python-generators/

You can turn it into a list however and load it in memory to see the content as a list:

In [6]:
print(list(doc))

[I, have, an, awesome, cat, ., She, follows, me, through, the, house, everywhere, I, go, ., Her, name, is, shadow]


As with a list, we can also access each token individually using the item index:

In [15]:
# Select one single token by index
first_token = doc[0]
print("First token:", first_token)
second_token = doc[1]
print("Second token:", second_token)

First token: I
Second token: have


Please note that even though these tokens look like strings, they are not. Print just gives the print representation of the token. We can also ask for the 'type' in Python and it will tell you what kind of object it is:

In [8]:
for token in doc:
    print(token, "\t", type(token))

I 	 <class 'spacy.tokens.token.Token'>
have 	 <class 'spacy.tokens.token.Token'>
an 	 <class 'spacy.tokens.token.Token'>
awesome 	 <class 'spacy.tokens.token.Token'>
cat 	 <class 'spacy.tokens.token.Token'>
. 	 <class 'spacy.tokens.token.Token'>
She 	 <class 'spacy.tokens.token.Token'>
follows 	 <class 'spacy.tokens.token.Token'>
me 	 <class 'spacy.tokens.token.Token'>
through 	 <class 'spacy.tokens.token.Token'>
the 	 <class 'spacy.tokens.token.Token'>
house 	 <class 'spacy.tokens.token.Token'>
everywhere 	 <class 'spacy.tokens.token.Token'>
I 	 <class 'spacy.tokens.token.Token'>
go 	 <class 'spacy.tokens.token.Token'>
. 	 <class 'spacy.tokens.token.Token'>
Her 	 <class 'spacy.tokens.token.Token'>
name 	 <class 'spacy.tokens.token.Token'>
is 	 <class 'spacy.tokens.token.Token'>
shadow 	 <class 'spacy.tokens.token.Token'>


These `Token` objects have many useful *methods* and *attributes*, which we can get listed by using the Python function `dir()`.

In [32]:
dir(first_token)

['_',
 '__bytes__',
 '__class__',
 '__delattr__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__len__',
 '__lt__',
 '__ne__',
 '__new__',
 '__pyx_vtable__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__unicode__',
 'ancestors',
 'check_flag',
 'children',
 'cluster',
 'conjuncts',
 'dep',
 'dep_',
 'doc',
 'ent_id',
 'ent_id_',
 'ent_iob',
 'ent_iob_',
 'ent_kb_id',
 'ent_kb_id_',
 'ent_type',
 'ent_type_',
 'get_extension',
 'has_extension',
 'has_vector',
 'head',
 'i',
 'idx',
 'is_alpha',
 'is_ancestor',
 'is_ascii',
 'is_bracket',
 'is_currency',
 'is_digit',
 'is_left_punct',
 'is_lower',
 'is_oov',
 'is_punct',
 'is_quote',
 'is_right_punct',
 'is_sent_start',
 'is_space',
 'is_stop',
 'is_title',
 'is_upper',
 'lang',
 'lang_',
 'left_edge',
 'lefts',
 'lemma',
 'lemma_',
 'lex_id',
 'like_email',
 'li

We haven't really talked about attributes during this course, but while methods are operations or activities performed by that object, attributes are 'static' features or properties of objects. Methods are called using parantheses (as we have seen with `str.upper()`, for instance), while attributes are indicated without parentheses. We will see some examples below. In the case of spaCy tokens, attributes typically contain *annotations* of the token in the text.

You can find more detailed information about the token methods and attributes in the [documentation](https://spacy.io/api/token).

Notice that there are many attributes with double listings, one without and once with the suffix `_`. The attributes without `_` actually have numerical values that spaCy uses internally, whereas variants with `_` have the human readable rendering of the value in unicode. The internal numerical repesentations are used to store data more efficiently, whereas the readable values are only generated for rendering output.

In [22]:
for token in doc:
    print(token.lemma, token.lemma_,token.pos, token.pos_)

561228191312463089 -PRON- 95 PRON
14692702688101715474 have 100 VERB
15099054000809333061 an 90 DET
3240785716591152042 awesome 84 ADJ
5439657043933447811 cat 92 NOUN
12646065887601541794 . 97 PUNCT
561228191312463089 -PRON- 95 PRON
14462500713227930305 follow 100 VERB
561228191312463089 -PRON- 95 PRON
18216413589307435838 through 85 ADP
7425985699627899538 the 90 DET
9471806766518506264 house 92 NOUN
10957650314384693728 everywhere 86 ADV
561228191312463089 -PRON- 95 PRON
8004577259940138793 go 100 VERB
12646065887601541794 . 97 PUNCT
561228191312463089 -PRON- 90 DET
18309932012808971453 name 92 NOUN
10382539506755952630 be 100 VERB
9756944156339232386 shadow 92 NOUN


You can also use spacy.explain to find out more about certain labels:

In [24]:
# try out some more, such as NN, ADP, PRP, VBD, VBP, VBZ, WDT, aux, nsubj, pobj, dobj, npadvmod
spacy.explain("ADP")

'adposition'

## Sentence splitting & tokenization
spaCy performs sentence splitting for you. The information is stored in the attribute **sents** of `Doc` (of type *spacy.tokens.doc.Doc*).
Each `Doc` contains a sequence of `Token` objects, i.e., this is where the output from the tokenizer is found. The token itself can be accessed using the attribute **text**. Each `Doc` instance will also have an index over the tokens to group them into sentences. We can iterate over these sentence indexes and get the tokens from each sentence in sequence. This will access the text sentence by sentence.

In [37]:
doc = nlp("I have an awesome cat. She follows me through the house everywhere I go. Her name is shadow")

In [25]:
sentences=doc.sents
for sentence in sentences:
    print('NEXT SENTENCE')
    print(sentence)
    for token in sentence:
        print(token.text)

NEXT SENTENCE
I have an awesome cat.
I
have
an
awesome
cat
.
NEXT SENTENCE
She follows me through the house everywhere I go.
She
follows
me
through
the
house
everywhere
I
go
.
NEXT SENTENCE
Her name is shadow
Her
name
is
shadow


## Lemmatization
The output from the lemmatizer is stored in the attribute **lemma_** of each `Token' object.

In [26]:
doc = nlp("I have awesome cats")

In [27]:
cat_token = doc[3]
print(cat_token.text, cat_token.lemma_)

cats cat


## Part of speech tagging

The output from the part of speech tagger is stored:
* in the attribute **pos_** of each `Token` object: The simple part-of-speech tag
* in the attribute **tag_** of each `Token object: The detailed part-of-speech tag ([Penn Treebank POS tagset](https://www.ling.upenn.edu/courses/Fall_2003/ling001/penn_treebank_pos.html))

In [28]:
doc = nlp("I have awesome cats")

In [29]:
cat_token = doc[3]
print(cat_token.text, cat_token.pos_, cat_token.tag_)
spacy.explain(cat_token.tag_)

cats NOUN NNS


'noun, plural'

## Dependency parsing
The output of the dependency parser can only be accessed by combining the information from multiple attributes. Let's look at an example:

In [30]:
doc = nlp("Autonomous cars shift insurance liability toward manufacturers")

spaCy has a special function *displacy* to display data. We first need to import it and next can apply it to the *doc* object. There are various paramters that can be said. You can read more in the spaCy documentation: https://spacy.io/usage/visualizers. Now, we choose the *style* 'dep' for depenedency.

In [31]:
from spacy import displacy
displacy.render(doc, jupyter=True, style='dep')

We observe that each token has a dependency relation with at least one other token. For example:
* **cars** has an **amod** relation with **autonomous**
* the main verb **shift** has an **nsubj** relation with **cars**

If you want to know what these relations mean, you can use **spacy.explain**

In [32]:
spacy.explain('amod')

'adjectival modifier'

spaCy makes use of the terms **child** and **head** in their dependency parsing output.
* a relation is always in one direction from a **child** to a **head**, e.g., *autonomous* is the child of *cars*
* a head of a phrase can be the child of another token, e.g., *cars* is the child of *shift*
* a token without a head is the root of the text or sentence (often the main verb)

The following attributes are needed to access this information:
* **dep_** provides the syntactic relation, e.g., *nsubj*
* **head** provides the **head** of a `Token`, e.g., in the case of *autonomous* the head would be *cars*

In [33]:
doc = nlp("Autonomous cars shift insurance liability toward manufacturers")

In [34]:
autonomous_token = doc[0]
print(autonomous_token, autonomous_token.dep_, autonomous_token.head)

Autonomous amod cars


In [35]:
cars_token = doc[1]
print(cars_token, cars_token.dep_, cars_token.head)

cars nsubj shift


### save tree structure to SVG image

In [36]:
tree_structure = displacy.render(doc, jupyter=False, style='dep')

output_path = 'spacy_tree_structure.svg'
with open(output_path, 'w') as outfile:
    outfile.write(tree_structure)

## Named Entity Recognition
The output from the Named Entity Recognizer is stored in the attribute **ents** of `Doc`.
The attribute **label_** and an **ent** (of type *spacy.tokens.span.Span*) contains the named entity type.

In [37]:
text = """But Google is starting from behind. The company made a late push into hardware, and Apple’s Siri, available on iPhones, and Amazon’s Alexa software, which runs on its Echo and Dot devices, have clear leads in consumer adoption."""
doc = nlp(text)

In [38]:
displacy.render(doc, jupyter=True, style='ent')

In [39]:
for ent in doc.ents:
    print(ent.text, ent.label_)

Google ORG
Apple’s Siri ORG
iPhones ORG
Amazon ORG
Echo and Dot ORG


# End of this notebook